In [59]:
#importing the  libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [60]:
#reading the dataset:

df=pd.read_csv('dataset.csv')

In [61]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0


In [62]:
df.isnull().sum()

id                       0
gender                   0
age                      0
hypertension             0
heart_disease            0
ever_married             0
work_type                0
Residence_type           0
avg_glucose_level        0
bmi                   1462
smoking_status       13292
stroke                   0
dtype: int64

In [63]:
df.dtypes

id                     int64
gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

In [64]:
df['stroke'].value_counts()

stroke
0    42617
1      783
Name: count, dtype: int64

In [65]:
df.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,43400.000000,43400.000000,43400.000000,43400.000000,43400.000000,41938.000000,43400.000000
mean,36326.142350,42.217894,0.093571,0.047512,104.482750,28.605038,0.018041
std,21072.134879,22.519649,0.291235,0.212733,43.111751,7.770020,0.133103
min,1.000000,0.080000,0.000000,0.000000,55.000000,10.100000,0.000000
25%,18038.500000,24.000000,0.000000,0.000000,77.540000,23.200000,0.000000
50%,36351.500000,44.000000,0.000000,0.000000,91.580000,27.700000,0.000000
75%,54514.250000,60.000000,0.000000,0.000000,112.070000,32.900000,0.000000
max,72943.000000,82.000000,1.000000,1.000000,291.050000,97.600000,1.000000


In [66]:
from sklearn.impute import SimpleImputer

imputer=SimpleImputer(missing_values=np.nan,strategy='most_frequent')
df_imputed=imputer.fit_transform(df)

df_imputed=pd.DataFrame(df_imputed,columns=df.columns)

In [67]:
from sklearn.preprocessing import OneHotEncoder
categorical_cols=['gender','ever_married','work_type','Residence_type','smoking_status']

df_encoded=pd.get_dummies(df_imputed,columns=categorical_cols)

In [68]:
df_encoded.shape

(43400, 22)

In [69]:
df_imputed.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [70]:
df_imputed.groupby('stroke').size()

stroke
0    42617
1      783
dtype: int64

In [71]:
#upsampling 

from sklearn.utils import resample

df_majority =df_encoded[df_encoded['stroke']==0]
df_minority=df_encoded[df_encoded['stroke']==1]


df_minority_upsampled=resample(df_minority,replace=True,n_samples=len(df_majority),random_state=42)

df_final=pd.concat([df_majority,df_minority_upsampled])

In [72]:
df_final.shape


(85234, 22)

In [73]:
df_final.shape

(85234, 22)

In [74]:
type(df_final)

pandas.core.frame.DataFrame

In [75]:
df_final.groupby('stroke').size()

stroke
0    42617
1    42617
dtype: int64

In [76]:
#dependent and independent features:

X=df_final.iloc[:,:-1]
y=df_final.iloc[:,-1]

In [77]:
#splitting the data

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=42)


In [78]:
#fitting the model

from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(X_train,y_train) 

RandomForestClassifier()

In [79]:
#prediction

y_pred=model.predict(X_test)

In [80]:
#metrics:

from sklearn.metrics import accuracy_score,f1_score

accuracy=accuracy_score(y_pred,y_test)
f1=f1_score(y_pred,y_test)
print('The accuracy of the model is ',accuracy)
print('The f1 score of the model is',f1)

The accuracy of the model is  0.9999706693259811
The f1 score of the model is 0.9999081304547542
